In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from config import *
from cal_accuracy import *
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
# 加载模型并使用
loaded_model = xgb.Booster()
loaded_model.load_model("xgboost_model.model")

In [3]:
# test_data_path = r'E:\Python\Code\Mel_recognition\derive_MFCC_for_ML\output\test_mfcc_features.xlsx'
test_data_path = r'./input/80_mfcc_features.xlsx'
# test_data_path = r'./input/BX_mfcc_features.xlsx'
test_data = pd.read_excel(test_data_path)
X_test = test_data[used_feature_columns].values
y_test = test_data['score']
dtest = xgb.DMatrix(X_test)
y_pred = loaded_model.predict(dtest)

In [4]:
y_pred

array([4.776188 , 5.1395283, 4.351365 , 5.6734014, 5.256562 , 3.8593256,
       4.4258914, 3.2893384, 4.922205 , 4.1608357, 4.5311165, 4.443876 ,
       5.624592 , 5.1075344, 5.4319086, 5.452294 , 6.4140944, 4.266591 ,
       4.328673 , 6.826655 , 6.603935 , 5.153317 , 6.1467543, 6.126073 ,
       7.019328 , 5.903679 , 5.77272  , 5.97805  , 5.9953604, 6.2089095,
       3.9785883, 4.008742 , 4.2825437, 4.970814 , 4.2075524, 5.2583847,
       3.6790996, 6.0078325, 5.464769 , 6.431015 , 3.906386 , 5.589835 ,
       4.474938 , 4.2986474, 5.525362 , 4.916931 , 4.290227 , 4.3644586,
       5.5108237, 5.0944157, 4.9915376, 4.7803617, 4.768901 , 4.663909 ,
       4.5155745, 4.3508425, 4.8890862, 3.6157756, 3.1133025, 3.5797791,
       3.4686577, 4.427801 , 4.578123 , 4.8204603, 4.762848 , 4.828783 ,
       4.4724092, 5.013092 , 4.8228054, 2.7113628, 5.4629855, 4.978195 ,
       5.458784 , 4.1526403, 4.0300756, 4.7072763, 4.883318 , 5.581065 ,
       4.8893404, 5.4115176], dtype=float32)

In [5]:
contrast = pd.DataFrame()
contrast['true'] = y_test
contrast['predict'] = y_pred
contrast['difference'] = abs(y_pred - y_test)
contrast['file_name'] = test_data['file_name']
contrast

,true,predict,difference,file_name
0,4.575,4.776188,0.201188,ba1-M301-1731.wav
1,5.100,5.139528,0.039528,ba2-M302-0823.wav
2,5.150,4.351365,0.798635,ba3-M303-0743.wav
3,3.150,5.673401,2.523401,ba4-M304-1030.wav
4,5.125,5.256562,0.131562,ba5-M306-0708.wav
...,...,...,...,...
75,5.925,4.707276,1.217724,ns11-N601-1050.wav
76,5.200,4.883318,0.316682,ns12-N602-0859.wav
77,4.250,5.581065,1.331065,ns13-N603-1527.wav
78,5.750,4.889340,0.860660,ns14-N604-1029.wav


In [6]:
threshold = 1
count = (contrast['difference'] < threshold).sum()
acc = count / len(contrast)
acc

0.6875

In [7]:
avg_diff = contrast['difference'].sum() / len(contrast)
# avg_percent = avg_diff / 8
avg_percent = avg_diff / 5
(1 - avg_percent)

0.8452851999998092

In [8]:
r2 = r2_score(contrast['true'], contrast['predict'])
r2

0.1938328383647676

In [9]:
predict_class_list = []
true_class_list = []
for i in range(len(contrast)):
    predict_class, true_class = classify(contrast.predict[i], contrast.true[i])
    predict_class_list.append(predict_class)
    true_class_list.append(true_class)

acc = accuracy(predict_class_list, true_class_list)
acc

0.8

In [10]:
mse = mean_squared_error(contrast['predict'], contrast['true'])
rmse = np.sqrt(mse)
mae = mean_absolute_error(contrast['predict'], contrast['true'])
mse, rmse, mae

(0.9168985512852348, 0.9575481978914873, 0.7735740000009537)